# Expressions régulières

Alexandre Bovet

UNamur et UCLouvain

alexandre.bovet@unamur.be


### Strings et expressions régulières
Plusieurs méthodes très utiles:
- `index()`
- `split()`
- `count()`
- `replace()`
- `strip()`
- …

Mais limitées à des cas simples!

Si méthodes pour les string conviennent, alors on les utilise!

Mais si utilisation devient complexe => Expressions régulières!


### Expressions régulières

- Méthode standardisée et très puissante pour rechercher et remplacer des motifs/patterns complexes de caractères dans des strings

- Syntaxe peut être *spéciale* (mais Python permet de les commenter)

- Exemple: numéros de téléphone 0XX.XX.XX.XX :
       `^0[0-9]{2}(.[0-9]{2}){4}`
       
"Some people, when confronted with a problem, think “I know, I’ll use regular expressions.” Now they have two problems." — Jamie Zawinski 
       
Ressources:
- https://docs.python.org/3/library/re.html
- https://docs.python.org/3/howto/regex.html#regex-howto

#### Exemple: standardiser des adresses
Solution avec les méthodes des strings:

In [ ]:
s = '100 NORTH MAIN ROAD' 
s.replace('ROAD', 'RD.')  

In [ ]:
s = '100 NORTH BROAD ROAD' 
s.replace('ROAD', 'RD.')

=> Problème! 

In [ ]:
s[:-4] + s[-4:].replace('ROAD', 'RD.') 

Solution pas robuste...

=> utilisation des expressions régulières (module `re`)

In [ ]:
import re 
re.sub('ROAD$', 'RD.', s) 
'100 NORTH BROAD RD.' 

-`sub()` remplace `ROAD` à la fin d’un String par `RD.` dans `s`
- -> `ROAD$` = expression régulière
- `$` : fin du String
- `^` : début du String
- caractères spéciaux: `. ^ $ * + ? { } [ ] \ | ( )`

Il faut penser à tous les cas possibles…

In [ ]:
#Problème: adresses ne se terminant pas par ROAD
s = '100 BROAD' 
re.sub('ROAD$', 'RD.', s)

In [ ]:
re.sub(r'\bROAD$', 'RD.', s)

ROAD est à la fin : `$`

ET mot complet: `\b` MAIS `\` caractère spécial 
- –> `\\b` sinon Pyhon ne considère pas `\`
- –>  `r'String'` :  raw String, i.e  considère tout! 


In [ ]:
#Problème: ROAD ne se trouve pas à la fin du String
s = '100 BROAD ROAD APT. 3' 
re.sub(r'\bROAD$', 'RD.', s) 

In [ ]:
re.sub(r'\bROAD\b', 'RD.', s)

Solution: `ROAD` se trouve quelque part dans le String
- –>  `\bROAD\b`


#### Exemple: chiffres romains

7 caractères:

- I = 1
- V = 5
- X = 10
- L = 50
- C = 100
- D = 500
- M = 1000

Règles:

- Addition : II, III, VI, VII, VIII, XI
- Répétition (3x max) : MMM, CC, XXX
- Soustraction : IV, IX, XC, CM
- V, L et D ne peuvent être répéter: X et pas VV
- Lecture de gauche à droite
- Ordre a de l’importance : CD ≠ DC



##### Vérification des milliers

In [ ]:
pattern = '^M?M?M?$'

Pattern a 3 parties:
- `^M` : `M` au début. Si absent: peut commencer par n’importe quoi!
- `?`  : correspond à 1 caractère  optionnel -> 3 `M` optionnels à la suite
- `$`  : avec `^` le pattern doit correspondre au String complet!

`search(pattern, str)` : 
- teste une correspondance  entre `str` et `pattern`.
- Si oui : retourne un objet, sinon: retourne `None` 

In [ ]:
re.search(pattern, 'M')

In [ ]:
re.search(pattern, 'MM') 

In [ ]:
re.search(pattern, 'MMM')

In [ ]:
re.search(pattern, 'MMMM')

In [ ]:
re.search(pattern, '')

String vide correspond car les `M` sont tous optionnels!

##### Vérification des centaines
- Plus compliqué que les milliers:

- 100 = C
- 200 = CC
- 300 = CCC
- 400 = CD
- 500 = D
- 600 = DC
- 700 = DCC
- 800 = DCCC
- 900 = CM

**Patterns:**
- CM
- CD
- 0 à 3 C (0 quand pas de centaines)
- D suivi de 0 à 3 C

=> 2 derniers peuvent être combinés:
- D optionnel suivi de 0 à 3 C




In [ ]:
pattern = '^M?M?M?(CM|CD|D?C?C?C?)$'

- Débute comme le précédent
- **`(x|y|z)` : 3 patterns mutuellement exclusif**
- `?` : caractère spécial => `x?` match 0 ou 1 répétition de `x` (caractère `x` facultatif)

In [ ]:
re.search(pattern, 'MCM')

In [ ]:
re.search(pattern, 'MD')

In [ ]:
re.search(pattern, 'MMMCCC')

**OK** :
- M optionnels au début
- CM / D / CCC / C est un pattern dans ()


In [ ]:
re.search(pattern, 'MCMC')

**not OK** :
- M optionnel au début, OK
- CM, OK
- MAIS `$` **not OK** à cause du dernier C  sans correspondance car `(x|y|z)` = patterns MUTUELLEMENT exclusifs!


In [ ]:
re.search(pattern, '')

##### Vérifications des dizaines

In [ ]:
pattern = '^M?M?M?(CM|CD|D?C?C?C?)(XC|XL|L?X?X?X?)$' 

In [ ]:
re.search(pattern, 'MCMXL')

In [ ]:
re.search(pattern, 'MCML') 

In [ ]:
re.search(pattern, 'MCMLX')

In [ ]:
re.search(pattern, 'MCMLXXX') 

In [ ]:
re.search(pattern, 'MCMLXXXX')

##### Vérification des unités

In [ ]:
pattern = '^M?M?M?(CM|CD|D?C?C?C?)(XC|XL|L?X?X?X?)(IX|IV|V?I?I?I?)$'

Améliorer la lisibilité du code en utilisant `{n|m}`
- `X{n,m}` = pattern `X` répété entre n et m fois


In [ ]:
pattern = '^M{0,3}$'

In [ ]:
re.search(pattern, 'M')

In [ ]:
re.search(pattern, 'MM')

In [ ]:
re.search(pattern, 'MMM')

In [ ]:
re.search(pattern, 'MMMM')

##### Chiffre romains: Pattern final

In [ ]:
pattern = '^(M{0,3})(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})$'

- `(...)` indique le début et la fin d'un groupe

In [ ]:
re.search(pattern, 'MDLV')

In [ ]:
re.search(pattern, 'MMDCLXVI')

In [ ]:
re.search(pattern, 'MMMDCCCLXXXVIII')

In [ ]:
re.search(pattern, 'I')

In [ ]:
match = re.search(pattern, 'MMMDCCCLXXXVIII')

In [ ]:
match.groups()

#### Expressions régulières documentées

In [ ]:
pattern = '''
    ^                   # beginning of string
    (M{0,3})            # thousands - 0 to 3 Ms
    (CM|CD|D?C{0,3})    # hundreds - 900 (CM), 400 (CD), 0-300 (0 to 3 Cs),
                        #            or 500-800 (D, followed by 0 to 3 Cs)
    (XC|XL|L?X{0,3})    # tens - 90 (XC), 40 (XL), 0-30 (0 to 3 Xs),
                        #        or 50-80 (L, followed by 0 to 3 Xs)
    (IX|IV|V?I{0,3})    # ones - 9 (IX), 4 (IV), 0-3 (0 to 3 Is),
                        #        or 5-8 (V, followed by 0 to 3 Is)
    $                   # end of string
    ''' 
re.search(pattern, 'MCMLXXXIX', re.VERBOSE)

doit utiliser le paramètre `re.VERBOSE`

#### Exemple: numéros de téléphones US
- Quand on a une correspondance, on peut en extraire des morceaux!

- 800-555-1212
- 800 555 1212
- 800.555.1212
- (800) 555-1212
- 1-800-555-1212
- 800-555-1212-1234
- 800-555-1212x1234
- 800-555-1212 ext. 1234
- work 1-(800) 555.1212 #1234

##### Essai 1

In [ ]:
phonePattern = re.compile(r'^(\d{3})-(\d{3})-(\d{4})$') 

- `compile()` : rendre la recherche plus efficace
- `(x)` = groupe
- `\d` = chiffre 0-9
- `{k}` = exactement k répétitions

In [ ]:
phonePattern.search('800-555-1212').groups() 

In [ ]:
phonePattern.search('800-555-1212-1234') 

In [ ]:
phonePattern.search('800-555-1212-1234') .groups()

##### Essai 2: ajout des extensions

In [ ]:
phonePattern = re.compile(r'^(\d{3})-(\d{3})-(\d{4})-(\d+)$') 
phonePattern.search('800-555-1212-1234').groups()

- `+` : au moins 1 répétition

In [ ]:
phonePattern.search('800 555 1212 1234') #no match

In [ ]:
phonePattern.search('800-555-1212') # no match

##### Essai 3: séparateurs entre groupes

In [ ]:
phonePattern = re.compile(r'^(\d{3})\D+(\d{3})\D+(\d{4})\D+(\d+)$')

- `\D` : tout sauf un chiffre

In [ ]:
phonePattern.search('800 555 1212 1234').groups()

In [ ]:
phonePattern.search('800-555-1212-1234').groups()

In [ ]:
phonePattern.search('80055512121234')

In [ ]:
phonePattern.search('800-555-1212')

##### Essai 4: séparateurs facultatifs entre groupes

In [ ]:
phonePattern = re.compile(r'^(\d{3})\D*(\d{3})\D*(\d{4})\D*(\d*)$')

- `*` :  0 ou + répétitions

In [ ]:
phonePattern.search('80055512121234').groups()

In [ ]:
phonePattern.search('800.555.1212 x1234').groups()

In [ ]:
phonePattern.search('800-555-1212').groups()

In [ ]:
phonePattern.search('(800)5551212 x1234')

**not OK** pour `()` entourant 800

##### Essai 5: Ignorer  ce qui se trouve au début

In [ ]:
phonePattern = re.compile(r'(\d{3})\D*(\d{3})\D*(\d{4})\D*(\d*)$')

- Pas de `^`  => Recherche d’une correspondance, mais pas nécessairement au début!

In [ ]:
phonePattern.search('work 1-(800) 555.1212 #1234').groups() 

In [ ]:
phonePattern.search('800-555-1212').groups() 

In [ ]:
phonePattern.search('80055512121234').groups() 

##### Etape finale: expression régulière documentée

In [ ]:
phonePattern = re.compile(r'''
                # don't match beginning of string, number can start anywhere
    (\d{3})     # area code is 3 digits (e.g. '800')
    \D*         # optional separator is any number of non-digits
    (\d{3})     # trunk is 3 digits (e.g. '555')
    \D*         # optional separator
    (\d{4})     # rest of number is 4 digits (e.g. '1212')
    \D*         # optional separator
    (\d*)       # extension is optional and can be any number of digits
    $           # end of string
    ''', re.VERBOSE)

In [ ]:
phonePattern.search('work 1-(800) 555.1212 #1234').groups()

### Résumé
- `^`       : début de string
- `$`       : fin d’un string
- `\b`      : caractère vide au début ou à la fin d’un mot
- `\d`      : caractère numérique
- `\D`      : caractère non-numérique
- `x?`      : caractère x facultatif (0 ou 1 répétition de x)
- `x*`      : 0 ou plus répétitions de x
- `x+`      : au moins 1 répétiton de x
- `x{m,n}`  : entre m et n répétitions de x
- `(a|b|c)` : exactement a OU b OU c
- `(x)`     : un groupe à récupérer via méthode groups()
- `[xyz]` : groupe de caractères, e.g. [a-d] = caractères a, b, c, d
- `.`    : tous les caractères sauf \n
- `\s`   : caractères d’espacements



### Exercice: générer le pluriel de mots anglais
Règles (simplifiées):

|Fin du mot| Action|
|----------|-------|
|S, X, Z   | + ES  |
|Lettre différente de {a,e,i,o,u,d,g,k,p,r,t} + H| + ES|
|Lettre différente de {aeiou} + Y | - Y + IES |
|Tout le reste | +S|

##### Substitutions
`sub('in','out','str')`: remplace toutes les occurrences de in par out dans str 

In [ ]:
re.sub('[abc]', 'o', 'Mark')

In [ ]:
re.sub('[abc]', 'o', 'rock')

In [ ]:
re.sub('[abc]', 'o', 'caps')

##### négation
`[^x]` = tout sauf x => négation


##### définissez la function plural qui prend un `noun` en argument et retourne son pluriel

In [ ]:
def plural(noun):
    pass